Использование networkx и matplotlib для построения графа, узлы - это друзья. Количество узлов условно до 20. Узлы обозначены "Имя". Входные данные загружаются из файла json.

Исходный файл с друзьями получаю по API VK.

In [ ]:
import numpy as np
import pandas as pd
import sys
import requests as req
import json
from lxml import objectify
import time
from random import randint
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
token_nom= 'your_s token'
app_nom = 'your_s number app'
user_id = 'your_s id'

In [ ]:
url = 'https://api.vk.com/method/friends.get?v=5.131'
url_get_fr = 'https://api.vk.com/method/friends.getMutual?v=5.131'
url_get_info = 'https://api.vk.com/method/users.get?v=5.131'

params = {'user_id': user_id,
         'fields':'id, first_name, last_name,',
         'access_token': token_nom}
r = req.get(url, params = params)
otvet = r.json()
# otvet
print(otvet['response']['count'])
result_list = {'source':[], 'id': [], 'name': []}
friendships = []

for i in otvet['response']['items'][:20]:
    result_list['id'].append(i['id'])
    result_list['name'].append(i['first_name']+' '+i['last_name'])
    result_list['source'].append('старт')
    friendships.append(('старт', i['first_name']+' '+i['last_name']))

df = pd.DataFrame(data=result_list)

with open('Friends', 'w', encoding='utf-8') as file:
    df.to_json(file, force_ascii=False)


for i in range(4):
    
    params_get_fr = {
    'user_id':user_id,
    'source_uid':df.iloc[i]['id'],
    'target_uid':df.iloc[i+1]['id'],
    'access_token': token_nom
    }
    
    time.sleep(randint(1,3))
    resp_friend_mut = req.get(url_get_fr, params=params_get_fr)
    print('\n')

    for j in resp_friend_mut.json()['response']:
        
        params_get_user = {
        'user_ids': j,
        'access_token': token_nom 
        }
        time.sleep(randint(1,3))
        resp_get_user = req.get(url_get_info, params = params_get_user)
        print(resp_get_user.json()['response'][0]['first_name']+' '+resp_get_user.json()['response'][0]['last_name']) 
        friendships.append((df.iloc[i]['name'], resp_get_user.json()['response'][0]['first_name']+' '+resp_get_user.json()['response'][0]['last_name']))
        friendships.append((df.iloc[i+1]['name'], resp_get_user.json()['response'][0]['first_name']+' '+ resp_get_user.json()['response'][0]['last_name']))
    
    print('друг : ',df.iloc[i]['id'],df.iloc[i]['name'])
    print('следующий друг : ', df.iloc[i+1]['id'],df.iloc[i+1]['name'])
    print('список : ', resp_friend_mut.json()['response'])
    
    
G=nx.Graph()
G.add_edges_from(friendships)
nx.draw(G, with_labels=True, node_color='g', node_size = 30, cmap = plt.cm.RdYlBu, font_size = 10)
plt.show()
